In [9]:
import pickle
import os
import numpy as np
import pandas as pd
import torch
import cv2
import matplotlib.pyplot as plt
from scipy.stats import beta
from sklearn.metrics import f1_score
from utils.notebook_utils import calibration_curve, estimate_dice_li, dice # cali-curve needed?
from utils.notebook_utils import clahe_equalized, process_image, inference
from utils.notebook_utils import scale_temp, apply_temperature, eval_func
from utils.notebook_utils import scale_temp_dice, eval_func_dice
from utils.notebook_utils import get_ensemble, error_fn, measure_uncertainty
from utils.notebook_utils import pass_and_log, softmax, pixel_uncertainty
from utils.model_definition import FR_UNet
from glob import glob
import bunch
import yaml
from munch import munchify
import pickle as pkl

In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [11]:
with open('paths.yaml', 'r') as file0:
        cfg_paths = munchify(yaml.load(file0, Loader=yaml.FullLoader))
        file0.close()

<u> Data: </u>

In [3]:
# Load the data
root = cfg_paths.FIVES

test_x = sorted(glob(os.path.join(root, 'test/Original/*')))
test_y = sorted(glob(os.path.join(root, 'test/Ground truth/*')))

In [4]:
# reconstruct train and val split
train_x = sorted(glob(os.path.join(root, 'train/Original/*')))
train_y = sorted(glob(os.path.join(root, 'train/Ground truth/*')))

validation_split = .2
indices = list(range(len(train_x)))
split = int(np.floor(validation_split * len(train_x)))
np.random.seed(23) # manually confirmed with the training script
np.random.shuffle(indices)

train_indices, val_indices = indices[split:], indices[:split]

val_x = train_x[:split]
val_y = train_y[:split]

<u> Models: </u>

In [7]:
# Deep Ensembles
ensemble_paths = 'trained/*' # hacky way to get all the models

list_models = []
for model_path in glob(ensemble_paths):
    list_models.append(model_path)

ensemble_models = get_ensemble(list_models, dropout=False, device=device)

<u> Forward Passes and DICE Estimation Looping over Full Test Set: </u>

In [8]:
ensemble_dict = pass_and_log(ensemble_models, device=device, x_paths=test_x, y_paths=test_y)

In [10]:
with open('cache/ensemble_dict.pkl', 'wb') as file:
    pkl.dump(ensemble_dict, file)
    file.close()

Validation Set

In [12]:
ensemble_dict_val = pass_and_log(ensemble_models, device=device, x_paths=val_x, y_paths=val_y)

In [ ]:
with open('cache/ensemble_dict_val.pkl', 'wb') as file2:
    pkl.dump(ensemble_dict_val, file2)
    file2.close()

Post Process Ensemble Forward Passes: Compute Mean Logits and Predictions

### Temperature Scaling as in Li et al. (2022)

In [15]:
T_dice_ensemble = scale_temp_dice(ensemble_dict_val['mean logits'], np.mean(ensemble_dict_val['true dice']))

In [17]:
T_dice_ensemble

1.4591796874999994

Apply Temperature Factor

In [20]:
temperature_dict_ens = apply_temperature(T_dice_ensemble, ensemble_dict['mean logits'], np.array(ensemble_dict['true dice']).mean(axis=0))